<a href="https://colab.research.google.com/github/MuzaffarAli13/OpenAI-Agents-Sdk-Topics-Governor-House/blob/main/Runner_Hooks_8_August_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install openai-agents SDK


In [ ]:
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 872.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.1/166.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.8 MB/s eta 0:00:00


## Make your Notebook capable of running asynchronous functions.

In [ ]:
import nest_asyncio
nest_asyncio.apply()

## Config

In [ ]:
from agents import (
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
)
from google.colab import userdata

In [ ]:
gemini_api_key = userdata.get("GEMINI_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)


In [ ]:
from agents import set_default_openai_client, set_tracing_disabled
set_default_openai_client(external_client)
set_tracing_disabled(True)

## Learning Agent LifeCycle

In [ ]:
# Imports
import asyncio
import random
from typing import Any

from pydantic import BaseModel

from agents import Agent, RunContextWrapper, RunHooks, Runner, Tool, function_tool


In [ ]:
# Simple Run Hooks - tracks EVERYTHING in the run
class MyRunnerHooks(RunHooks):
    async def on_agent_start(self, context, agent):
        # print(f">> {agent.name} started.")
        print("on_agent_start in runner hooks:", agent.name)

    async def on_agent_end(self, context, agent, output):
        # print(f">> {agent.name} finished.")
        print("on_agent_end in runner hooks:", agent.name)

    async def on_tool_start(self, context, agent, tool):
        # print(f">> Agent {agent.name} call {tool.name}.")
        print("on_tool_start in runner hook", agent.name, tool.name)

    async def on_tool_end(self, context, agent, tool, result):
        # print(f">> Tool {tool.name} excecuted.")
        print("on_tool_end in runner hook", agent.name, tool.name)
        print("Tool Result:", result)



In [ ]:
# Simple tools
@function_tool
def check_order():
    """Check order status"""
    return "Order #123 is on the way!"

@function_tool
def check_refund():
    """Check refund status"""
    return "Refund of $50 processed!"


# Create TWO agents no hooks on agent
agent1 = Agent(
    name="Order Agent",
    instructions="Check order status",
    tools=[check_order],
    model=model,
    # NOTE: NO hooks on this agent
)

agent2 = Agent(
    name="Refund Agent",
    instructions="Check refund status",
    tools=[check_refund],
    model=model,
    # NOTE: NO hooks on this agent
)


In [ ]:
async def main():

    print('=' * 50)
    print("RUN HOOKS DEMO")
    print('=' * 50)
    print()

    # Create runner hooks
    run_hooks = MyRunnerHooks()

    # Run 1: Order Agent
    print("📦 Checking order...")
    result1 = await Runner.run(
        starting_agent=agent1,
        input="Check my order",
        hooks=run_hooks,
    )
    print(f"\n[ANSWER] {result1.final_output}")


    # Run 2: Refund Agent (same hooks can track this too!)
    print("\n💰 Checking refund...")
    result2 = await Runner.run(
        starting_agent=agent2,
        input="Check my refund",
        hooks=run_hooks, # <- SAME HOOKS ON THIS RUN
    )

    print(f"\n[ANSWER] {result2.final_output}")

asyncio.run(main())


RUN HOOKS DEMO

📦 Checking order...
>> Order Agent started.
>> Agent Order Agent call check_order.
>> Tool check_order excecuted.
Tool Result: Order #123 is on the way!
>> Order Agent finished.

[ANSWER] OK. Order #123 is on the way!


💰 Checking refund...
>> Refund Agent started.
>> Agent Refund Agent call check_refund.
>> Tool check_refund excecuted.
Tool Result: Refund of $50 processed!
>> Refund Agent finished.

[ANSWER] OK. I've checked your refund status. The refund of $50 has been processed.



## Make a `main agent` that will handoff to `order agent` and `refund agent`.
## Implement `on_handoff` hook